In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_fixed_cost_runs(acq, dim):
    runs = api.runs(path="ziv-scully-group/Gittins for Bayesian Optimization", filters={
        "sweep": "jzoa5n1q", 
        "config.lengthscale": 0.1, 
        "config.amplitude": 1, 
        "config.kernel": "matern52", 
        "config.dim": dim,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [7]:
acquisition_functions = {
    'ExpectedImprovement':'ExpectedImprovement',
    'ThompsonSampling':'ThompsonSampling', 
    # 'KnowledgeGradient':'KnowledgeGradient',
    'Gittins_Lambda_01':'Gittins_Lambda01',
    'Gittins_Lambda_001':'Gittins_Lambda001',
    'Gittins_Lambda_0001':'Gittins_Lambda0001',
    'Gittins_Step_Divide2':'Gittins_Step_Divide2',
    'Gittins_Step_Divide5':'Gittins_Step_Divide5',
    'Gittins_Step_Divide10':'Gittins_Step_Divide10',
    'Gittins_Step_EIpu':'Gittins_Step_EIPC' , 
    }
dimensions = [16]

In [3]:
grouped_runs = {(a,d): load_bayesian_regret_fixed_cost_runs(a,d) for a in acquisition_functions.keys() for d in (dimensions)}

100%|███████████████████████████████████████████| 16/16 [00:15<00:00,  1.04it/s]


In [6]:
grouped_runs

{('ExpectedImprovement',
  16): [({'dim': 16,
    'seed': 12,
    'kernel': 'matern52',
    'policy': 'ExpectedImprovement',
    'amplitude': 1,
    'lengthscale': 0.1,
    'num_rff_features': 1280,
    'draw_initial_method': 'sobol',
    'input_normalization': False},
   {'cumulative cost': [0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44,
     45,
     46,
     47,
     48,
     49,
     50,
     51,
     52,
     53,
     54,
     55,
     56,
     57,
     58,
     59,
     60,
     61,
     62,
     63,
     64,
     65,
     66,
     67,
     68,
     69,
     70,
     71,
     72,
     73,
     74,
     75,
     76,
     77,
     78,
     79,
 

In [8]:
for a in acquisition_functions.keys():
    for d in dimensions:
        config_and_metrics_per_seed = grouped_runs[a,d]

        cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
        best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T
        global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed])

        regret_per_seed = global_optimum_per_seed - best_observed_per_seed

        regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
        regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
        regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)

        output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)

        np.savetxt(f"bayesian_regret/BayesianRegret_FixedCost_d{d}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.